In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import sys
import platform
import logging
import argparse
from datetime import datetime, date, timedelta
import pendulum
import pandas as pd
import numpy as np

In [2]:
# Add parent directory to path
repo_path = '/home/notebook/prod/nelson/dview-naarni-data-platform'
sys.path.append(os.path.join(repo_path, 'tasks'))

# Import necessary files and its respective functions
from common.db_operations import connect_to_trino, fetch_data_for_day, write_df_to_iceberg,drop_table,execute_query
from common.optimizer_logic import optimize_dataframe_memory

# Import business logic functions
from biz_logic.energy_mileage.energy_mileage_daily_v0 import energy_mileage_stats ,impute_odometer_readings

from biz_logic.energy_consumption.energy_consumption_report import energy_consumption_stats

# Configure basic logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# Print the Python version being used
print(f"Using Python version: {platform.python_version()}")

Using Python version: 3.11.13


In [3]:
# ---- CLI Argument Parser ----
def is_running_in_jupyter():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' in get_ipython().config:
            return True
    except Exception:
        pass
    return False


def parse_arguments():
    parser = argparse.ArgumentParser(description="Process facts-prod bizlogic data.")
    parser.add_argument("--date", type=str, help="Date to process in YYYY-MM-DD format (IST).")
    parser.add_argument("--yesterday", action="store_true", help="Process yesterday's data (overrides --date if both are provided)")

    if is_running_in_jupyter():
        class DefaultArgs:
            date = None
            yesterday = False
        return DefaultArgs()

    return parser.parse_args()


# ---- Date Handling (Fixed Timezone Logic) ----
def get_target_date(args):
    """Determine which date's data to process (IST-based)."""
    tz = pendulum.timezone("Asia/Kolkata")

    if args.date:
        target_date = pendulum.parse(args.date, tz=tz).date()
    elif args.yesterday:
        target_date = pendulum.now(tz).subtract(days=1).date()
    else:
        target_date = pendulum.now(tz).date()

    logging.info(f"Script running at {pendulum.now(tz)} (IST)")
    logging.info(f"Extracting data for {target_date}")
    return target_date


def process_data(conn, df: pd.DataFrame, table_name: str, schema: str = "facts_prod", push_to_db: bool = True, partition_by: list = None):
    # Process data related energy-mileage relation
    df_res = df
    if not df_res.empty:
        if push_to_db:
            write_df_to_iceberg(conn, df_res, table_name, schema, partition_by=partition_by)
            logging.info(f"✅ Processing and write for {table_name} complete with partitioning by {partition_by}.")
        else:
            logging.info(f"✅ Processing for {table_name} complete. Data not pushed to database.")
    else:
        logging.info("Processed DataFrame is empty. No data to write.")
    return df_res

In [4]:
# ---- Test configuration ----
TEST_DATE = "2025-09-29"  # Change this to the date you want to test
SOURCE_TABLE = "can_parsed_output_100"
COLUMNS_TO_FETCH = [
    '"id"','"timestamp"','"dt"',
    'at_timezone("timestamp", \'Asia/Kolkata\') AS IST',
    '"BAT_SOC"',
    '"Bat_Voltage"',
    '"BrakePedalPos"', 
    '"Vehicle_speed_VCU"',
    '"Total_Battery_Current"',
    '"GUN_Connection_Status"',
    '"OdoMeterReading"',
    '"Gear_Position"',
    '"ignitionstatus"',
    '"Vehiclereadycondition"',
    "'motor_rpm'",
    "'motor_torque'",
    '"Chargingcontactor1positive"',
    '"Chargingcontactor1negative"',
    '"Chargingcontactor2positive"',
    '"Chargingcontactor2negative"'
]

In [5]:
# --------------------
# Main execution logic
# --------------------
def main(start_date_str: str = None, end_date_str: str = None, schema: str = "facts_prod", push_to_db: bool = True, partition_by: list = None):
    conn = connect_to_trino()
    vehicle_ids_for_report = []    
    
    if conn:
        try:
            # Determine the date range to process
            if start_date_str and end_date_str:
                logging.info(f"📅 Processing explicit date range: {start_date_str} to {end_date_str}")
                start_date = date.fromisoformat(start_date_str)
                end_date = date.fromisoformat(end_date_str)
                date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]
            else:
                # Default to processing yesterday's data in IST
                logging.info("📅 No date range specified, defaulting to yesterday's data in IST")
                args = parse_arguments()
                yesterday_ist = get_target_date(args)                
                date_range = [yesterday_ist]
                logging.info(f"📅 Processing date: {yesterday_ist.isoformat()}")

            for single_date in date_range:
                date_str = single_date.isoformat()
                logging.info(f"▶️ Starting daily report job for {date_str}")

                # Example 2: Call the function with specific vehicle IDs
                logging.info("\n--- Processing specific vehicle IDs ---")
                # vehicle_ids_for_report = ['3', '16', '18', '19']
                df_raw_specific = fetch_data_for_day(conn, date_str, COLUMNS_TO_FETCH, SOURCE_TABLE,schema, vehicle_ids_for_report)

                
                if not df_raw_specific.empty:
                    # df_processed_energy_consumption = process_data(conn,energy_consumption_stats(df_raw_specific))

                    # Process data for power_consumption_report
                    process_data(
                        conn,
                        energy_consumption_stats(df_raw_specific),
                        "energy_consumption_report",
                        schema, 
                        push_to_db, 
                        partition_by
                    )

                    # # Process data for energy_mileage_report
                    process_data(
                        conn, 
                        energy_mileage_stats(df_raw_specific), 
                        "energy_mileage_report", 
                        schema, 
                        push_to_db, 
                        partition_by
                    )
                else:
                    logging.info("Raw DataFrame is empty. No processing needed.")

        except Exception as e:
            logging.critical(f"❌ A critical error occurred in the main script: {e}")

        finally:
            logging.info("🔒 STEP 5: Closing Trino connection...")
            conn.close()
            logging.info("✅ STEP 5: Connection closed.")
    else:
        logging.critical("❌ Failed to establish a database connection. Exiting.")

    # return df_processed_energy_consumption, df_processed_energy_mileage

In [6]:
# main(start_date_str='2025-09-25', end_date_str='2025-09-28',push_to_db=False, partition_by= ['dt'])
main(start_date_str='2025-10-25', end_date_str='2025-10-25',push_to_db=False)
# main(push_to_db=False)

2025-10-27 08:29:23 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-10-27 08:29:23 - INFO - ✅ STEP 1: Connected to Trino
2025-10-27 08:29:23 - INFO - 📅 Processing explicit date range: 2025-10-25 to 2025-10-25
2025-10-27 08:29:23 - INFO - ▶️ Starting daily report job for 2025-10-25
2025-10-27 08:29:23 - INFO - 
--- Processing specific vehicle IDs ---
2025-10-27 08:29:23 - INFO - 📥 STEP 2a: Validating and fetching data for 2025-10-25 (IST)...
2025-10-27 08:29:23 - INFO - ⚙️ Executing query...
2025-10-27 08:29:58 - INFO - ✅ Query executed successfully with results!
2025-10-27 08:29:58 - INFO - 📊 STEP 2b: Timestamp range in fetched data:
2025-10-27 08:29:58 - INFO - 🔍 Query window (UTC): 2025-10-24 18:30:00 → 2025-10-25 18:30:00
2025-10-27 08:29:58 - INFO - 🔍 Query window (IST): 2025-10-25 00:00:00 → 2025-10-26 00:00:00
2025-10-27 08:29:58 - INFO - ✅ STEP 2d: Data fetching for 2025-10-25 (IST) completed, Rows fetched: 833378
2025-10-27 08:29:58 - INFO - ⚙️ STEP 3a: Starting data processing.

In [7]:
wait

NameError: name 'wait' is not defined

In [ ]:
# df_processed_energy_mileage = main(start_date_str='2025-08-25', end_date_str='2025-08-25',schema = "facts_dev",push_to_db = False)

In [ ]:
df_processed_energy_mileage

In [ ]:
min(df_raw[df_raw.Total_Battery_Current.abs()<1000].Total_Battery_Current)

In [ ]:
display(df_raw.head())
len(df_raw)

In [ ]:
df_raw.sort_values(by='IST', ascending= True)

In [ ]:
df_raw.to_csv('high_regen.csv')

In [ ]:
display(df_energy_mileage.head())

In [ ]:
len(df_raw)

In [ ]:
# df_energy_mileage[df_energy_mileage.mileage_kwh_per_km > 0.9]

In [ ]:
wait 

In [ ]:
conn = connect_to_trino()
drop_table(conn,"energy_mileage_daily_v1","facts_dev")
conn.close()

In [ ]:
conn = connect_to_trino()

# Step 1: Create a new table with DATE type
query1 = '''
CREATE TABLE facts_dev.energy_mileage_daily_v1 (
    vehicle_id VARCHAR,
    date DATE,
    dist_travelled_km DOUBLE,
    consumed_energy_kwh DOUBLE,
    regen_energy_kwh DOUBLE,
    idling_energy_kwh DOUBLE,
    idling_duration_mins DOUBLE,
    net_energy_kwh DOUBLE,
    mileage_kwh_per_km DOUBLE,
    is_inactive BOOLEAN,
    dt TIMESTAMP(6)
)
'''
execute_query(conn, query1, return_results=False)

In [ ]:
# Step 2: Insert data with converted date field using date_parse
query2 = '''
INSERT INTO facts_dev.energy_mileage_daily_v1
SELECT 
    vehicle_id,
    date_parse(date, '%Y-%m-%d') AS date,  -- Parse varchar as DATE
    dist_travelled_km,
    consumed_energy_kwh,
    regen_energy_kwh,
    idling_energy_kwh,
    idling_duration_mins,
    net_energy_kwh,
    mileage_kwh_per_km,
    is_inactive,
    dt
FROM facts_prod.energy_mileage_daily_v1
'''
execute_query(conn, query2, return_results=False)

# Continue with the rest of the steps as above...

In [ ]:
conn = connect_to_trino()

# Step 1: Create temporary table with distinct records
# Create a new table with unique records
query1 = """
SELECT DISTINCT * FROM facts_prod.energy_mileage_report
where date(date) = date('2025-09-28')
"""
df = execute_query(conn, query1)

# # Step 3: Reinsert the distinct records
# query3 = '''
# INSERT INTO facts_prod.energy_mileage_report
# SELECT * FROM temp_distinct_records;
# '''
# execute_query(conn, query3)

# # Step 4: Clean up
# query4 = '''
# DROP TABLE temp_distinct_records;
# '''
# execute_query(conn, query4)

conn.close()
df

In [ ]:
conn = connect_to_trino()
# Step 2: Delete all records for that date
query2 = '''
DELETE FROM facts_prod.energy_consumption_report
WHERE date(start_time) = date('2025-09-24')
'''
execute_query(conn, query2, return_results=False)

In [ ]:
conn = connect_to_trino()
# Step 2: Delete all records for that date
query2 = '''
DELETE FROM facts_prod.energy_mileage_report
WHERE date(date) = date('2025-09-24')
'''
execute_query(conn, query2, return_results=False)

In [ ]:
conn = connect_to_trino()
drop_table(conn,"energy_mileage_daily_fixed","facts_prod")
drop_table(conn,"energy_mileage_daily_v1","facts_prod")
conn.close()

In [ ]:
conn = connect_to_trino()
query1 = '''
ALTER TABLE facts_prod.energy_mileage_report_fixed RENAME TO facts_prod.energy_mileage_report
'''
execute_query(conn, query1, return_results=False)
conn.close()

24 hours = 86400 datapoints
24 hours = 40k data point

40k datapoint = 40% datapoints are usable